In [2]:
train_dir = '../../datasets/xview/train'
test_dir = '../../datasets/xview/test'
suppl_dir = '../../datasets/xview/tier3'
tertiary_dir = '../../datasets/spacenet'


In [3]:
from xv.dataset import get_instances, get_mask

In [4]:
from glob import glob
files = glob(train_dir + '/labels/*pre_disaster.json')
instances = get_instances(files)

100%|██████████| 2799/2799 [00:27<00:00, 103.03it/s]


In [11]:
import numpy as np
def split_save(instance):
    image = np.array(Image.open(instance['file_name']))
    h, w, _ = image.shape

    polys = [a['segmentation'] for a in instance['annotations']]
    msk = get_mask(polys, w, h)

    hm, wm = h//2, w//2
    split_im = image[:hm, :wm], image[:hm, wm:], image[hm:, :wm], image[hm:, wm:]
    split_msk = msk[:hm, :wm], msk[:hm, wm:], msk[hm:, :wm], msk[hm:, wm:]

    base_img_path = instance['file_name'].replace('images', 'images_split')
    base_msk_path = instance['file_name'].replace('images', 'masks_split').replace('png', 'npy')

    for ix, (i, m) in enumerate(zip(split_im, split_msk)):
        Image.fromarray(i).save(base_img_path.replace('pre_disaster', f'{ix}_pre_disaster'))
        np.save(base_msk_path.replace('pre_disaster', f'{ix}_pre_disaster'), m)
        
len(instances)